In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
import numpy as np
import noise

from matplotlib import pyplot as plt
from matplotlib import colors as col

from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display
import ipywidgets as wdg

from plotly import graph_objs as plgo
from plotly import offline as ploff
ploff.init_notebook_mode(connected=False)

In [ ]:
base = np.linspace(0, 5, 100)
x, y = np.meshgrid(base, base)
final_func = np.vectorize(lambda x, y: noise.pnoise2(x, y, octaves=4))
land = final_func(x, y)

water = np.zeros(land.shape)
water[45:56,45:56] = 0.5 - land[45:56,45:56]

In [ ]:
def plot(x, y, land, water):
#     mask = ~water.astype(bool)
    mask = water < 0.0001
    layer = land + water
    layer[mask] = None
    ploff.iplot(
        plgo.Figure({
            'data': [
                plgo.Surface({
                    'name': 'land',
                    'x': x,
                    'y': y,
                    'z': land,
                    'colorscale': [(0.0, '#F9EBEA'), (1.0, '#641E16')],
                }),
                plgo.Surface({
                    'name': 'water',
                    'x': x,
                    'y': y,
                    'z': layer,
                    'colorscale': [(0.0, '#154360'), (1.0, '#85C1E9')],
                    'showscale': False,
                })
            ],
            'layout': plgo.Layout({
                'title': 'Simulation',
                'width': 800,
                'height': 800,
                'scene': plgo.Scene({
                    'zaxis': plgo.ZAxis({
                        'range': [-1, 1.5]
                    })
                })
            }),
        })
    )

In [ ]:
def upper_plot(land, water, Timestep):
    mask = water[Timestep] < 0.0001
    layer = land + water[Timestep]
    layer[mask] = None
    land_cm = col.LinearSegmentedColormap.from_list(
        'land',
        [(110/255, 44/255, 0), (246/255, 221/255, 204/255)][::-1]
    )
    water_cm = col.LinearSegmentedColormap.from_list(
        'water',
        [(27/255, 79/255, 144/255), (214/255, 234/255, 248/255)]
    )
    fig = plt.figure(figsize=(14,8))
    ax = fig.add_subplot(111)
    ax.imshow(land, cmap=land_cm, vmin=np.min(land), vmax=np.max(land))
    ax.imshow(layer, cmap=water_cm, vmin=np.min(land), vmax=np.max(land))
    plt.show()

In [ ]:
def simple_automata_step(terra,water,alpha):
    assert(terra.shape==water.shape)
    levels = terra+water
    # Get the level transfer:
    transfer_x = levels[:,:-1]-levels[:,1:]
    transfer_x = np.minimum(water[:,:-1],transfer_x)
    transfer_x = np.maximum(-water[:,1:],transfer_x)
    #
    transfer_y = levels[:-1,:]-levels[1:,:]
    transfer_y = np.minimum(water[:-1,:],transfer_y)
    transfer_y = np.maximum(-water[1:,:],transfer_y)
    # Update water levels:
    nwater = water.copy()
    nwater[:,:-1] -= transfer_x*alpha
    nwater[:,1:]  += transfer_x*alpha
    nwater[:-1,:] -= transfer_y*alpha
    nwater[1:,:]  += transfer_y*alpha
    return nwater

In [ ]:
size = 1000
alpha = 0.2

w = np.zeros((size, *land.shape))
w[0,:,:] = simple_automata_step(land, water, alpha)
for i in log_progress(range(size-1), every=1, name='Timesteps'):
    w[i+1,:,:] = simple_automata_step(land, w[i,:,:], alpha)

In [ ]:
wdg.interact(upper_plot, Timestep=(0, size-1, 1), water=wdg.fixed(w), land=wdg.fixed(land))